# 🤺 Fence demo

This notebook demonstrates the use of various classes in this package. The core intent of this package is to provide a framework for interacting with language models in a way that is modular, extensible, and easy to use. The core classes are:
- `PromptTemplate`: A template for a prompt that can be rendered with a dictionary of variables (or using keywords). It wraps around jinja2.Template, but adds some additional functionality.
- `LLM`: A wrapper around a language model. 
- `Link`: Atomic LLM interaction wrapper. Takes in a PromptTemplate and a language model. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `TransformationLink`: A wrapper around a `function` that takes a dictionary of variables and returns a dictionary of variables. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `Chain`: A collection of links that are invoked in the right order based on the input and output keys for each Link.
- `LinearChain`: A sequence of links that are invoked in the order they are passed in.

Through these classes, we can interact with LLMs in varying degrees of complexity:
1. Just call the model directly
2. Use a `PromptTemplate` for reusability and abstraction
3. Use a `Link` for atomic LLM interaction
4. Use a `Chain` to execute a collection of `Links`

Below, we'll go through each of these levels of complexity and provide examples.

**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [14]:
from pathlib import Path
current_dir = Path('.').resolve().parents[0]
import sys
sys.path.append(str(current_dir))

In [15]:
%load_ext autoreload
from fence.models.bedrock.claude import ClaudeInstant
from fence.models.bedrock.claude import Claude35Sonnet
from fence.models.openai import GPT4omini
from fence.templates.string import StringTemplate
from fence.templates.messages import MessagesTemplate, Messages, Message
from fence.links import Link, TransformationLink
from fence.chains import Chain, LinearChain

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## ⚙️ Setting up

In [16]:
# Get our model
model = Claude35Sonnet(source='demo_notebook', region='us-east-1')
model = GPT4omini(source='demo_notebook')

### 🪨 Level 1 - Just call the damn thing

In [17]:
# Use the invoke method to call the model
model.invoke('Why is the sky blue?')

"The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it interacts with molecules and small particles in the air. Sunlight, or white light, is made up of different colors, each with different wavelengths. Blue light has a shorter wavelength compared to other colors.\n\nAs sunlight passes through the atmosphere, the shorter wavelengths of blue light are scattered in all directions more than the longer wavelengths, such as red or yellow light. This scattering causes the blue light to be more prevalent in the sky when we look up, creating the blue appearance that we associate with a clear day.\n\nDuring sunrise and sunset, the sun is lower on the horizon, and sunlight travels through a thicker layer of the atmosphere. This increased distance causes more scattering of shorter wavelengths and allows the longer wavelengths (reds and oranges) to dominate, resulting in the beautiful colors we see during those times."

In [18]:
# Just call the damn thing
model('Why is the sky blue?')

"The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it interacts with air molecules and small particles. Sunlight is composed of many colors, each with different wavelengths. Blue light has a shorter wavelength than colors like red or yellow.\n\nAs sunlight passes through the atmosphere, the shorter wavelengths (blue and violet) are scattered in all directions by the gases and particles in the air. Although violet light is scattered even more than blue light, our eyes are more sensitive to blue light and some of the violet light is absorbed by the ozone layer. This results in the sky predominantly appearing blue during the day.\n\nAt sunrise and sunset, the sky can take on shades of red, orange, and pink. This is because the sunlight passes through a greater thickness of the atmosphere, scattering most of the shorter blue wavelengths and allowing the longer red and orange wavelengths to dominate."

### 🔨 Level 2 - Use a PromptTemplate

In [19]:
# Initialize a prompt template
prompt_template = StringTemplate('Why is the sky {color}?')
print(prompt_template)

StringTemplate: Why is the sky {color}?


In [20]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky red?


In [21]:
prompt_template_test = StringTemplate('Why is the sky {color}?')
print(prompt_template_test.render({'color': 'blue'}))

Why is the sky blue?


In [22]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = StringTemplate('Why is the sky {color}?')
prompt_template_grass = StringTemplate('Why is the grass {color}?')
prompt_template_dress = StringTemplate('I like a dress with {pattern}.')
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

StringTemplate: Why is the sky {color}? Why is the grass {color}? I like a dress with {pattern}.
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [23]:
# You can customize the separator
base_template = StringTemplate('Why is the sky {color}?', separator=' FUNKY TOWN ')
additional_template = StringTemplate('Why is the grass {color}?')
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


In [24]:
# You can also use the MessagesTemplates for more complex prompts
messages = Messages(
    system='Respond in a {tone} tone',
    messages= [
        Message(role="user", content="Why is the sky {color}?"),
        # Equivalent to Message(role='user', content=Content(type='text', text='Why is the sky blue?'))
        # But Content can also be an image, etc.
    ]
)
messages_template = MessagesTemplate(
    source=messages
)
print(messages_template)
print(f"\nRendered: {messages_template.render(tone='sarcastic', color='blue')}")

MessagesTemplate: messages=[Message(role='user', content='Why is the sky {color}?')] system='Respond in a {tone} tone'

Rendered: messages=[Message(role='user', content='Why is the sky blue?')] system='Respond in a sarcastic tone'


### 🧠 Level 3 - Use Links (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [25]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, model=model)
print(link)

Link: <['color']> -> <state>


In [26]:
# Invoke it
link(color='blue') # Or, equivalently, link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue due to a phenomenon known as Rayleigh scattering. This occurs when sunlight interacts with the Earth's atmosphere, which is made up of various gases and particles.\n\nSunlight, or white light, is composed of many colors, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow. When sunlight enters the atmosphere, it collides with molecules and small particles, scattering the shorter blue wavelengths in all directions more effectively than the longer wavelengths.\n\nAs a result, when we look up at the sky, we see more of this scattered blue light than any other color, making the sky appear predominantly blue during the day. At sunrise and sunset, the light passes through more of the atmosphere, which scatters shorter wavelengths even more and allows the longer wavelengths (reds and oranges) to predominate, creating those beautiful colors at those times."}

In [27]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, model=model, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight passes through the Earth's atmosphere, it collides with molecules and small particles in the air. Sunlight, or white light, is made up of various colors, each with different wavelengths. Blue light has a shorter wavelength than other colors, such as red or yellow.\n\nBecause of its shorter wavelength, blue light is scattered in all directions more effectively than longer wavelengths when it interacts with the air molecules. As a result, when we look up at the sky, we see more of this scattered blue light. \n\nDuring sunrise and sunset, the light has to pass through a greater thickness of the atmosphere, scattering shorter wavelengths out of the direct line of sight and allowing longer wavelengths like red and orange to dominate the view. This is why the sky can appear more red or orange during those times.",
 'intermediate': "The sky appears blue primarily due to a phenomenon called 

In [28]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, model=model, name='sky')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight reaches Earth's atmosphere, it is made up of various colors of light, which correspond to different wavelengths. Blue light has a shorter wavelength and is scattered in all directions by the gases and particles in the atmosphere more effectively than colors with longer wavelengths, such as red or yellow.\n\nDuring the day, when the sun is high in the sky, the blue light is scattered across the sky, making it appear predominantly blue to our eyes. At sunrise and sunset, the sun's light has to travel through more of the Earth's atmosphere. This longer path scatters the shorter blue wavelengths even more and allows the longer wavelengths, like red and orange, to dominate the sky's colors, creating the beautiful hues often observed at these times."}

In [29]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(input: dict):
    return f"{input['X']} and {input['Y']}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

{'state': 'Hello and World', 'C': 'Hello and World'}

### 🚀 Level 4 - Use Chains (lol again)

In [36]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=StringTemplate(
        "What's the opposite of {A}? Reply with a few words max."
    ),
    name = 'opposite',
    output_key="X",
)
link_synonym = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word."
    ),
    name='synonym',
    output_key="Y",
)
link_poem = Link(
    template=StringTemplate(
        "Write a poem about {state}. Return only the poem, beginning with the title."
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(model=model, links=[link_opposite, link_synonym, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
from pprint import pprint
pprint(result)

{'A': 'A police officer',
 'B': 'Hopeful',
 'C': 'A criminal. and Optimistic.',
 'X': 'A criminal.',
 'Y': 'Optimistic.',
 'Z': '**The Optimistic Outlaw**\n'
      '\n'
      'In shadows deep where whispers dwell,  \n'
      'A rogue with tales to spin and tell,  \n'
      'With a heart that dances, wild and free,  \n'
      'Chasing dreams beneath the moonlit tree.  \n'
      '\n'
      'His hands may bear the scars of night,  \n'
      'Yet in his eyes, a spark so bright,  \n'
      'He steals not for greed or gain,  \n'
      'But for the thrill of breaking the chain.  \n'
      '\n'
      'With every heist of stolen glance,  \n'
      'He crafts a world where hearts can dance,  \n'
      'A twist of fate, a chance to rise,  \n'
      'For even thorns can wear a disguise.  \n'
      '\n'
      'In alleyways where shadows creep,  \n'
      'He plants the seeds of hope to seep,  \n'
      'A laugh, a jest, a wink, a nod,  \n'
      'Turning mischief into a facade.  \n'
      '\n'
    

In [31]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=StringTemplate(
        "Capitalize this word: {A}. Only respond with the capitalized version",
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word.", 
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=StringTemplate(
        "Combine {X} and {Y} and {C} in a meaningful sentence.", 
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(model=model, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['C', 'Y', 'X']> -> <Z>]

In [32]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'B', 'A', 'C'}. Missing: {'C'}


In [33]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})
print(result['state'])

The optimistic police officer knelt down to pet the friendly dog, believing that their connection could help bridge the gap between the community and law enforcement.


In [34]:
# Cycles are not allowed
link_up = Link(
    template=StringTemplate(
        "Capitalize this word: {up}. Only respond with the capitalized version",
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {down}. Reply with one word.", 
    ),
    name='down',
    output_key="up",
)
chain = Chain(model=model, links=[link_up, link_b])
try:
    chain.run(input_dict={"up": "happy"})
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
